In [1]:
import numpy as np
from scqubits import Fluxonium
import matplotlib.pyplot as plt


def calculate_energy(flxs, EJ, EC, EL, cutoff=50):
    fluxonium = Fluxonium(EJ, EC, EL, flux=0.0, cutoff=cutoff, truncated_dim=10)
    spectrumData = fluxonium.get_spectrum_vs_paramvals(
        "flux", flxs, evals_count=10
    )

    return spectrumData.energy_table

# load data

In [2]:
import h5py

def load_data(file_path):
    with h5py.File(file_path, "r") as file:
        data = file["Data"]['Data']
        if data.shape[2] == 1: # 1D data,
            x_data = data[:, 0, 0][:]
            y_data = None
            z_data = data[:, 1, 0][:] + 1j * data[:, 2, 0][:]
        else:
            x_data = data[:, 0, 0][:]
            y_data = data[0, 1, :][:]
            z_data = data[:, 2, :][:] + 1j * data[:, 3, :][:]
    return z_data, x_data, y_data

def convert2max_contrast(Is: np.ndarray, Qs: np.ndarray):
    # calculate the covariance matrix
    cov = np.cov(Is, Qs)

    # calculate the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(cov)

    # sort the eigenvectors by decreasing eigenvalues
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # rotate the data
    data = np.vstack([Is, Qs])
    data_rot = np.dot(eigenvectors.T, data)

    return data_rot[0], data_rot[1]

spectrum, fpts, flxs = load_data("qub_flux_dep.hdf5")
fpts /= 1e9 # convert to GHz

sps =[]
for i in range(spectrum.shape[1]):
    sp, _ = convert2max_contrast(spectrum[:, i].real, spectrum[:, i].imag)
    sps.append(sp)
spectrum = np.array(sps).T

spectrum = spectrum - np.median(spectrum, axis=0)
spectrum = np.abs(spectrum)


if flxs[0] > flxs[-1]: # Ensure that the fluxes are in increasing
    flxs = flxs[::-1]
    spectrum = spectrum[:, ::-1]
if fpts[0] > fpts[-1]: # Ensure that the frequencies are in increasing
    fpts = fpts[::-1]
    spectrum = spectrum[::-1, :]

In [ ]:
plt.imshow(spectrum, aspect='auto', origin='lower', extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))

In [4]:
center = -0.0029338
halfp = 0.00436

cflxs = (flxs - center) / (2*halfp) + 0.5

In [ ]:
plt.imshow(spectrum, aspect='auto', origin='lower', extent=(cflxs[0], cflxs[-1], fpts[0], fpts[-1]))
plt.axvline(0.5, color='r')
plt.axvline(1.0, color='r')

In [6]:
flxs = cflxs

In [27]:
from scipy.ndimage import gaussian_filter1d

def spectrum_analyze(flxs, fpts, amps, ratio):
    # use guassian filter to smooth the spectrum
    amps = gaussian_filter1d(amps, 3, axis=0)
    amps = amps - np.median(amps, axis=0)
    amps = np.abs(amps)
    amps /= np.std(amps)

    # plot max point and min point of each row
    max_ids = np.argmax(amps, axis=0)
    maxs = amps[max_ids, np.arange(amps.shape[1])]

    # max points
    max_masks = maxs < ratio
    fs = fpts[max_ids]
    fs[max_masks] = np.nan
    mask = ~np.isnan(fs)

    return flxs[mask], fs[mask]

def remove_close_points(flxs, fs, dist_ratio):
    # remove some close points
    mask = np.ones(len(flxs), dtype=bool)
    t_d2 = np.sqrt((flxs[-1] - flxs[0])**2 + (fs[-1] - fs[0])**2) * dist_ratio
    prev = 0
    for i in range(1, len(flxs)):
        d_flx = flxs[i] - flxs[prev]
        d_fs = fs[i] - fs[prev]
        d2 = np.sqrt(d_flx**2 + d_fs**2)
        if d2 < t_d2:
            mask[i] = False
        else:
            prev = i

    return flxs[mask], fs[mask]

s_flxs, s_fs = spectrum_analyze(flxs, fpts, spectrum, 5.0)
# s_flxs, s_fs = remove_close_points(s_flxs, s_fs, 0.02)

In [28]:
# manual remove some points
remove_idxs = [9, 22, 23, 24, 25, 26]
s_flxs = np.delete(s_flxs, remove_idxs)
s_fs = np.delete(s_fs, remove_idxs)

In [ ]:
plt.imshow(spectrum, aspect='auto', origin='lower', extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))
plt.scatter(s_flxs, s_fs, color='r', s=3)

In [ ]:
import h5py as h5

with h5.File("data/fluxonium_2.h5", "r") as file:
    h_flxs = file["flxs"][:]  # type: ignore
    h_params = file["params"][:]  # type: ignore
    h_energies = file["energies"][:]  # type: ignore

transitions = [(0, 1), (0, 2), (1, 3)]
def dist_to_curve(energies, s_fs):
    fs = []
    for i, j in transitions:
        fs.append(energies[:, j] - energies[:, i])
    fs = np.array(fs).T # (n, m)

    dist = np.abs(fs - s_fs[:, None]) # (n, m)
    dist = np.nanmin(dist, axis=1) # (n, )

    return np.nansum(dist)

# find the closest index in energy to s_fs
sf_flxs = np.mod(s_flxs, 1.0)
d2 = (h_flxs[:, None] - sf_flxs[None, :])**2 # (m, m')
idxs = np.argmin(d2, axis=0) # (m', )

best_params = None
best_energy = None
best_dist = float("inf")
for i in range(h_params.shape[0]):
    energy = h_energies[i, idxs]

    dist = dist_to_curve(energy, s_fs)
    if dist < best_dist:
        best_dist = dist
        best_energy = h_energies[i]
        best_params = h_params[i]
print(best_params)


In [ ]:
f_energies = calculate_energy(flxs, *best_params, cutoff=60)

In [ ]:
plt.imshow(spectrum, aspect="auto", origin="lower", extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))
for i, j in transitions:
    plt.plot(flxs, f_energies[:, j] - f_energies[:, i], label=f"{i} -> {j}")
    # plt.plot(h_flxs, best_energy[:, j] - best_energy[:, i], label=f"{i} -> {j}")
plt.scatter(s_flxs, s_fs, c='r', s=3)
plt.ylim(fpts[0], fpts[-1])
plt.xlim(flxs[0], flxs[-1])
plt.legend()

# Scipy

In [33]:
EJb = (3.5, 6.5)
ECb = (0.3, 1.5)
ELb = (0.7, 2.5)

In [34]:
from scipy.optimize import minimize

def fit_spectrum(flxs, fpts, params):
    def loss_func(params):
        energies = calculate_energy(flxs, *params, cutoff=50)

        fs = []
        for i, j in transitions:
            fs.append(energies[:, j] - energies[:, i])
        fs = np.stack(fs, axis=1) # (n, m)
        dist = np.abs(fpts[:, None] - fs) # (n, m)
        loss_fs = np.nanmin(dist, axis=1) # (n,)

        return np.nansum(loss_fs)

    # aim to find current curve
    res = minimize(
        loss_func,
        params,
        bounds=(EJb, ECb, ELb),
        method="L-BFGS-B",
        options={"maxfun": 50},
    )

    return res.x

In [ ]:
# fit the spectrumData
sp_params = fit_spectrum(s_flxs, s_fs, params=best_params)

# print the results
print("Fitted params:", *sp_params)

In [ ]:
f_energies = calculate_energy(flxs, *sp_params, 60)

In [ ]:
# plt.pcolormesh(flxs, fpts, spectrum)
plt.imshow(spectrum, aspect='auto', origin='lower', extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))

for i, j in transitions:
    plt.plot(flxs, f_energies[:, j] - f_energies[:, i], label=f"{i}-{j}")

plt.scatter(s_flxs, s_fs, color="red")
plt.ylim(fpts[0], fpts[-1])
plt.legend()